### Autres caractéristiques

In [3]:
import pandas as pd
data_tweet = pd.read_csv("Tweets/french_tweets.csv")

texte = data_tweet["text"]

#tous les modules nécessaires :

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
import statistics
import re

tokenizer = nltk.RegexpTokenizer("([A-Z]\.[A-Z]?\.?[0-9]?|[0-9]+[,.][0-9]+|[cdjls]'|qu'|[\w'-]+|\S)")
nb_instances= 0
for i in texte:
    l=len(tokenizer.tokenize(i))
    nb_instances+=l

nb_instances3= 0
for i in texte:
    longueur_text=len(sent_tokenize(i))
    nb_instances3 += longueur_text

nb_carac = 0
for i in texte:
    longueur_carac = len(i)
    nb_carac += longueur_carac

In [4]:
# on reprend les comptes du début et on les adapte un peu

liste_mots = []
for i in texte:
    l=tokenizer.tokenize(i)
    for j in l:
        liste_mots.append(j)
# ici on fait une liste de tous les mots du corpus pour pouvoir l'utilisée ci-après

somme_longueur = 0
for i in liste_mots:
    longueur = len(i)
    somme_longueur += longueur
# ici, on fait la somme de toutes les longueurs des mots du corpus pour pouvoir obtenir la moyenne

moyenne_longueur = somme_longueur / len(liste_mots)

nb_carac = 0
for i in texte:
    longueur_carac = len(i)
    nb_carac += len(i)

nb_phrases= 0
for i in texte:
    longueur_text=len(sent_tokenize(i))
    nb_phrases += longueur_text

taille_phrases = 0
for i in texte:
    longueur=sent_tokenize(i)
    for j in longueur :
        titi = len(j)
        taille_phrases += titi

moyenne_longueur2 = taille_phrases / nb_phrases

In [6]:
import statistics

X_stylo = []#notre nouvelle matrice de description
for text in data_tweet["text"]:
    mots=tokenizer.tokenize(text)
    phrases = sent_tokenize(text)
    NB_phrases = len(phrases)
    NB_mots = len(mots)
    NB_caracteres = len(text)
    for x in mots :
        moyenne_taille_mots = statistics.mean([len(x)])
    moyenne_taille_phrases = NB_mots/nb_phrases
    caracteristiques = [NB_phrases, NB_mots, NB_caracteres, moyenne_taille_mots, moyenne_taille_phrases]
    X_stylo.append(caracteristiques)

In [7]:
nom_classes = ["Positif", "Négatif"]
y = data_tweet["label"]

X_train, X_test, y_train, y_test = train_test_split(X_stylo, y, test_size=0.3, random_state=0)
DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)
stats = precision_recall_fscore_support(y_test, y_pred)
print(stats)
report = classification_report(y_test, y_pred, target_names=nom_classes)
print(report)

y_spam = [1 for x in y_test]##une liste remplie de 1
report = classification_report(y_test, y_spam, target_names=nom_classes)
matrice_confusion = confusion_matrix(y_test, y_spam)
print(matrice_confusion)
print(report)

y_ham = [0 for x in y_test]##une liste remplie de 0
report = classification_report(y_test, y_ham, target_names=nom_classes)
matrice_confusion = confusion_matrix(y_test, y_ham)
print(matrice_confusion)
print(report)

[[136523  94691]
 [117040 109764]]
(array([0.53841846, 0.53686141]), array([0.59046165, 0.48395972]), array([0.56324042, 0.50903981]), array([231214, 226804]))
              precision    recall  f1-score   support

     Positif       0.54      0.59      0.56    231214
     Négatif       0.54      0.48      0.51    226804

    accuracy                           0.54    458018
   macro avg       0.54      0.54      0.54    458018
weighted avg       0.54      0.54      0.54    458018



/home/alexane.jouglar/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[     0 231214]
 [     0 226804]]
              precision    recall  f1-score   support

     Positif       0.00      0.00      0.00    231214
     Négatif       0.50      1.00      0.66    226804

    accuracy                           0.50    458018
   macro avg       0.25      0.50      0.33    458018
weighted avg       0.25      0.50      0.33    458018

[[231214      0]
 [226804      0]]
              precision    recall  f1-score   support

     Positif       0.50      1.00      0.67    231214
     Négatif       0.00      0.00      0.00    226804

    accuracy                           0.50    458018
   macro avg       0.25      0.50      0.34    458018
weighted avg       0.25      0.50      0.34    458018



In [8]:
#**On combine le BOW et le stylométrique**
V = CountVectorizer(ngram_range = (1,2) )
X = V.fit_transform(data_tweet["text"])
##on crée une sparse matrix avec notre X_stylo
from scipy.sparse import csr_matrix
sparse_stylo = csr_matrix(X_stylo)
print(sparse_stylo.shape[0])#NB lignes   -> instances
print(sparse_stylo.shape[1])#Nb colonnes -> caractéristiques

## on ale même nombre de ligne, on fait donc une conctaténation horizontale :
from scipy.sparse import hstack
X_fusion = hstack((X, sparse_stylo))

#résultats :

X_train, X_test, y_train, y_test = train_test_split(X_fusion, y, test_size=0.3, random_state=0)
DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
matrice_confusion = confusion_matrix(y_test, y_pred)
print(matrice_confusion)
stats = precision_recall_fscore_support(y_test, y_pred)
print(stats)
report = classification_report(y_test, y_pred, target_names=nom_classes)
print(report)

1526724
5
[[165676  65538]
 [ 63809 162995]]
(array([0.72194697, 0.71322304]), array([0.71654831, 0.71866016]), array([0.71923751, 0.71593128]), array([231214, 226804]))
              precision    recall  f1-score   support

     Positif       0.72      0.72      0.72    231214
     Négatif       0.71      0.72      0.72    226804

    accuracy                           0.72    458018
   macro avg       0.72      0.72      0.72    458018
weighted avg       0.72      0.72      0.72    458018



In [9]:
print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())
print("-"*10)

Bons résultats 328671
Erreurs: 129347
----------


## Autres classifieurs

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import warnings 
warnings.filterwarnings("ignore")

LR = LogisticRegression()
RDF = RandomForestClassifier()
SVC = SVC()
KN = KNeighborsClassifier()

Liste = [LR, RDF, SVC, KN]

for i in Liste:
    i = i.fit(X_train, y_train)
    y_pred = i.predict(X_test)
    matrice_confusion = confusion_matrix(y_test, y_pred)
    print(i)
    print(matrice_confusion)
    stats = precision_recall_fscore_support(y_test, y_pred)
    print(stats)
    report = classification_report(y_test, y_pred, target_names=nom_classes)
    print(report)
    print('Bons résultats %d' % (y_test == y_pred).sum())
    print('Erreurs: %d' % (y_test != y_pred).sum())
    print("-"*10)

LogisticRegression()
[[155388  75826]
 [ 54102 172702]]
(array([0.74174424, 0.69489957]), array([0.67205273, 0.76145923]), array([0.7051808 , 0.72665842]), array([231214, 226804]))
              precision    recall  f1-score   support

     Positif       0.74      0.67      0.71    231214
     Négatif       0.69      0.76      0.73    226804

    accuracy                           0.72    458018
   macro avg       0.72      0.72      0.72    458018
weighted avg       0.72      0.72      0.72    458018

Bons résultats 328090
Erreurs: 129928
----------


### En faisant varier les n-grammes

In [ ]:
for min_N in range(1, 2):
  for max_N in range(min_N, 4):
    V = CountVectorizer(ngram_range = (min_N, max_N), analyzer = "char")
    X = V.fit_transform(data_spam["text"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
    print(f"Ngram_range : ({min_N}, {max_N})")
    for i in Liste:
        clf = i.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print('%s classifier : %.4f'%(nom, score))

### Forêt aléatoire : zoom sur quelques paramètres

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
    # On fait la somme de tous les cas où la valeur dans y_test est bien trouvée dans y_pred
print('Bons résultats %d' % (y_test == y_pred).sum())
print('Erreurs: %d' % (y_test != y_pred).sum())

# ci-dessus : paramètres par défaut

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

for i in range(1,3):
    clf = RandomForestClassifier(max_depth=i, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # On fait la somme de tous les cas où la valeur dans y_test est bien trouvée dans y_pred
    print("Avec max_depth =", i)
    print('Bons résultats %d' % (y_test == y_pred).sum())
    print('Erreurs: %d' % (y_test != y_pred).sum())

### En supprimant la ponctuation et les stopword

In [ ]:
# rappels pour la cellule ci-après (ces éléments de codes ont été exécutés plus avant)
# LR = LogisticRegression()
# RDF = RandomForestClassifier()
# SVC = SVC()
# KN = KNeighborsClassifier()
# Liste = [LR, RDF, SVC, KN]

from nltk.corpus import stopwords

import re

def remove_ponctuation(chaine):
    ponctuations = [",", "'", '"', "-", "\."]
    for stopword in ponctuations:
        chaine = re.sub(f" {stopword} ", " ", chaine)
    return chaine

def remove_stopwords(chaine):
    final_stopwords_list = stopwords.words('english')
    s = chaine
    for stopword in final_stopwords_list:
        chaine = re.sub(f" {stopword} ", " ", chaine)
    return chaine

for pretraitement in ["stopwords", "ponctuation"]:
    if pretraitement =="stopwords":
        liste_pretraite = [remove_stopwords(j) for j in texte]
        X = V.fit_transform(liste_pretraite)
    elif pretraitement =="ponctuation":
        liste_titres_pretraite = [remove_ponctuation(j) for j in texte]
        X = V.fit_transform(liste_pretraite)    
    else:
        X = V.fit_transform(texte)
    for split_size in [0.3]: #[0.1, 0.2, 0.3, 0.9]:
        print(f"Split_size : {split_size}, Pretraitement: {pretraitement}")
    #découpage train VS test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state=0)
    ##classification
        for i in Liste:
            i = i.fit(X_train, y_train)
            y_pred = i.predict(X_test)
            matrice_confusion = confusion_matrix(y_test, y_pred)
            print(i)
            print(matrice_confusion)
            stats = precision_recall_fscore_support(y_test, y_pred)
            print(stats)
            report = classification_report(y_test, y_pred, target_names=nom_classes)
            print(report)
            print('Bons résultats %d' % (y_test == y_pred).sum())
            print('Erreurs: %d' % (y_test != y_pred).sum())
            print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
            print("-"*10)

from sklearn.metrics import accuracy_score

liste_pretraite = [remove_ponctuation(j) for j in texte]
X = V.fit_transform(liste_pretraite)    
for split_size in [0.3]:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state=0)

for i in Liste:
    i = i.fit(X_train, y_train)
    y_pred = i.predict(X_test)
    matrice_confusion = confusion_matrix(y_test, y_pred)
    print(i)
    print(matrice_confusion)
    stats = precision_recall_fscore_support(y_test, y_pred)
    print(stats)
    report = classification_report(y_test, y_pred, target_names=nom_classes)
    print(report)
    print('Bons résultats %d' % (y_test == y_pred).sum())
    print('Erreurs: %d' % (y_test != y_pred).sum())
    print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    print("-"*10)
#print(corpus_sans_stopwords)